# Exercice : Data Querying
Le but de cet exercice est de voire comment manipuler une source de données RDF déja existante, cette source est disponible sur GitHub, a travers ça l'objectif recherché est : 
- De mieux manipuler SPARQL
- De voire un cas d'application réel ou RDF a été utilisé pour annoter des données.

A chaque étape le but serra de satisfaire un besoin je vous laisse les sorties attendus dans le fichier de solution et je vous laisse la dernière sortie de la dernière cellule, si vous y arrivez normalement tout doit être correcte.

Première étape : Créer le graphe a partir du fichier et le charger, vous pouvez afficher le contenu du fichier en utilisant la fonction cat.

In [ ]:
import rdflib
graphe  = rdflib.Graph()
!cat gemet-backbone.rdf
graphe.parse("gemet-backbone.rdf")

Maintenant essayons d'explorer notre fichier donc la première étape serra de voire les concepts qui sont utilisés par l'ontologie.
Ici le but est de générer les noms de toutes les classes du fichier donc de trouver tous les tuples <Sujet, is a, Classe> en utilisant un filtre (pas de requète SPARQL pour l'instant)

In [ ]:
#Essayez d'identifier le namespace le plus utile a concerver 
ont_ns = "https://www.eionet.europa.eu/gemet/2004/06/gemet-schema.rdf#"
#Maintenant on va sérializer le fichier en turtle pour pouvoir le visualiser ce qui va vous aider dans l'écriture des requètes
graphe.serialize("gemet-backbone-turtle.rdf","turtle")

In [ ]:
!cat gemet-backbone-turtle.rdf

In [ ]:
"""
Pour vous aider je vous donne cet exemple de requètes qui récupérent tous les tuples du graphe rdf (et qui en garde
50) 
Les prefixes que j'ai pris ont été pris en utilisant ceux du fichier turtle.
J'ai aussi codé un affichage que vous pouvez réutiliser pour voire les réponses de vos requètes et que j'utiliserais
dans la solution.
"""

requete = """
prefix gemet: <http://www.eionet.europa.eu/gemet/2004/06/gemet-schema.rdf#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix skos: <http://www.w3.org/2004/02/skos/core#> 
prefix xml: <http://www.w3.org/XML/1998/namespace> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
SELECT ?s ?p ?o
WHERE {
?s ?p ?o
}
LIMIT 50
"""
rez = graphe.query(requete)
for i in rez:
    print("===")
    cpt = 0
    for j in i:
        print(cpt," - ",j)
        cpt = cpt + 1
    print("====")

Pour commencer prenons une classe, par exemple la classe Theme.

Et affichons 100 exemples de cette classe avec tous les tuples qui y sont associés

In [ ]:
requete = """
votre requète
"""
rez = graphe.query(requete)
for i in rez:
    print("===")
    cpt = 0
    for j in i:
        print(cpt," - ",j)
        cpt = cpt + 1
    print("====")

Première question que vous devez vous poser : 
Pourquoi cette requète ne donne aucune donnée littérale (String, int ou autre) ? 
Il suffit de cliquer sur un des liens que la requète renvoie pour comprendre pourquoi.

La réponse est que cette base d'annotations est un exemple ou les URI ne sont pas fictifs, ils référencent vraiment un contenu en ligne que le fichier RDF ne fait qu'annoter en précisant les ressources associées.

In [ ]:
"""
Afin de récupérer les titres des pages vers lesquelles pointent les URI je vous donne cette fonction que vous
permettra in-fine d'avoir une idée sur le contenu des pages.
"""
import requests
import re
def get_content(uri):
    r = requests.get(uri)
    text = str(r.content)
    L = re.findall("<title>(.*?)</title>",text)
    LL = []
    for i in L:
        LL.append(i.replace("\\n","").strip())
    return LL[0]

get_content("http://www.eionet.europa.eu/gemet/theme/20")

In [ ]:
"""
Maintenant on va extraire une partie des themes et tous les membres qui leurs sont reliés, on va les mettre dans un
graphe qu'on va enrichir au fur et a mesure.
"""
mon_ns = "https://monns/ontologie#"
skos_ns = "http://www.w3.org/2004/02/skos/core#"

graphe2 = rdflib.Graph()

requete = """
votre requète
"""
rez = graphe.query(requete)
for i in rez:
    print("===")
    cpt = 0
    for j in i:
        print(cpt," - ",j)
        cpt = cpt + 1
        
    #Instructions pour ajouter les themes qu'on a extrait, les concepts et les lier 
    print("====")

In [ ]:
graphe2.serialize("graphe_2.rdf","turtle")
!cat graphe_2.rdf

In [ ]:
#Je vous donne celle des thèmes inspirez vous en pour celle des concepts
requete = """
prefix gemet: <http://www.eionet.europa.eu/gemet/2004/06/gemet-schema.rdf#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix skos: <http://www.w3.org/2004/02/skos/core#> 
prefix xml: <http://www.w3.org/XML/1998/namespace> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
SELECT ?s 
WHERE {
?s rdf:type skos:theme .
}
"""
rez = graphe2.query(requete)
L = list(rez)
cpt = 1
for i in L:
    print("===")
    for j in i:
        print(cpt,"/",len(L)," - ",j)
    cpt = cpt + 1
    graphe2.add((i[0],rdflib.URIRef(mon_ns+"contenu"),rdflib.Literal(get_content(i[0]))))
    print("====")

In [ ]:
graphe2.serialize("graphe_2.rdf","turtle")
!cat graphe_2.rdf

In [ ]:
#Maintenons voyons les contenus qu'on a extrait
requete = """
prefix gemet: <http://www.eionet.europa.eu/gemet/2004/06/gemet-schema.rdf#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix skos: <http://www.w3.org/2004/02/skos/core#> 
prefix xml: <http://www.w3.org/XML/1998/namespace> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
prefix ns2: <https://monns/ontologie#> 
SELECT ?s ?c
WHERE {
?s ns2:contenu ?c .
}
"""
rez = graphe2.query(requete)
L = list(rez)
cpt = 1
for i in L:
    print("===")
    for j in i:
        print(cpt,"/",len(L)," - ",j)
    cpt = cpt + 1
    graphe2.add((i[0],rdflib.URIRef(mon_ns+"contenu"),rdflib.Literal(get_content(i[0]))))
    print("====")

In [ ]:
# A vous de faire la même chose pour les concepts qu'on trouve dans la base de données
requete = """
votre requète
"""
rez = graphe2.query(requete)
L = list(rez)
cpt = 1
#Limite pour éviter de faire trop de scrapping
limite = 100
for i in L:
    print("===")
    for j in i:
        print(cpt,"/",len(L)," - ",j)
    cpt = cpt + 1
    if(cpt == limite):
        break
    #Fonction qui ajoute au graphe comme celle d'en haut
    print("====")

In [ ]:
#Maintenons voyons les contenus qu'on a extrait
requete = """
prefix gemet: <http://www.eionet.europa.eu/gemet/2004/06/gemet-schema.rdf#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix skos: <http://www.w3.org/2004/02/skos/core#> 
prefix xml: <http://www.w3.org/XML/1998/namespace> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 
prefix ns2: <https://monns/ontologie#> 
SELECT ?s ?t ?c
WHERE {
?s ns2:contenu ?c .
?s rdf:type ?t
}
"""
rez = graphe2.query(requete)
for i in rez:
    print("===")
    cpt = 0
    for j in i:
        print(cpt," - ",j)
        cpt = cpt + 1
    print("====")

In [ ]:
#Maintenant l'objectif est de construire un dictionnaire theme : concepts réliés 
requete = """
requete
"""
D = {}
rez = graphe2.query(requete)
for i in rez:
    print("===")
    cpt = 0
    for j in i:
        print(cpt," - ",j)
        cpt = cpt + 1
    #votre fonction qui ajoute a la liste correspondant a la clef i[0] le mot i[1]
    print("====")

In [ ]:
import json
#On formatte tout ça en Json et on tri selon les clefs
print(json.dumps(D, sort_keys=True, indent=4))

In [ ]:
"""Maintenant imaginez qu'on veuille limiter notre recherche d'URI couteuse en mémoire aux 10 thémes
qui ont le plus de concepts
 - Revoyez comment on fait une aggrégation avec GroupBy
 - Checkez la documentation officielle pour voire des exemples en Sparql
et essayez d'écrire la requète qui fait ça.
"""
top = "10"
requete = """
votre requète qui retourne l'uri du theme en premier, puis le nombre de concepts en second
""".replace("%t",top)

rez = graphe.query(requete)
cpt = 1
D = {}
D["Theme"] = []
D["Classement"] = []
D["Nombre de termes"] = []

for i in rez:
    c=get_content(i[0])
    print("-",end="")
    D["Theme"].append(c)
    D["Classement"].append(cpt)
    D["Nombre de termes"].append(int(i[1]))
    cpt = cpt + 1
    
print("")
print("==TOP {}==".format(str(top)))
import pandas as pd
df = pd.DataFrame(D)
print(df)
print("=========")

print("==TOP {} STATS==".format(str(top)))
print(df.describe()["Nombre de termes"])
print("=========")